In [56]:
%load_ext autoreload
%autoreload 1
%aimport game

import game
import time
import random

game._build_row_tables()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
max_iters = 1000000
num_games = 100

game_results = []

for game_index in range(num_games):
    start_board = [
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
    ]
    bs = game.to_bitset(start_board)

    start_iter = time.time()

    for i in range(max_iters):
        bs = game.generate_tile(bs)
        action_space = game.get_action_space(bs)
        if action_space:
            bs = random.choice(action_space)(bs)
        else:
            break
        
    time_taken = time.time() - start_iter
    game_results.append({
        'iters': i,
        'max_tile': game.get_max_tile(bs),
        'time': time_taken,
    })


In [60]:
print(f'Games per second: {num_games / sum(result["time"] for result in game_results)}')
print(f'Average iterations: {sum(result["iters"] for result in game_results) / len(game_results)}')
print(f'Average max tile: {sum(result["max_tile"] for result in game_results) / len(game_results)}')

Games per second: 412.6854686995742
Average iterations: 116.16
Average max tile: 104.32
